In [165]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as torch_data
from torchsummary import summary
from torchvision import models
from pycocotools.coco import COCO
import torchvision.transforms as transforms
import json
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np

import pytorch_lightning as pl

import os, sys
sys.path.append(os.path.abspath("../"))

%load_ext autoreload
%autoreload 2


from models.base_line.model import BaseLine
from models.autoencoders.components import ConvBlock
from models.GANs.model import ConstrastDCGAN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
net = BaseLine(512, 0, 0)

In [118]:
torch.rand(4, 3, 128, 128)[:2].shape

torch.Size([2, 3, 128, 128])

In [119]:
batch = (
    torch.rand(4, 3, 128, 128),
    torch.rand(4, 3, 128, 128),
    torch.arange(4)
)

In [139]:
latent_dim = net._ae_forward(batch)["union"].flatten(1).shape[1:][0]
noise_dim = latent_dim//2

In [140]:
gan = ConstrastDCGAN(latent_dim, noise_dim)

In [141]:
gan.contrast_batch = torch.randn((4, latent_dim))

In [142]:
gan._get_fake_pred(torch.ones(4))

torch.Size([4, 3, 64, 64])


tensor([0.2788, 0.8317, 0.9472, 0.7412], grad_fn=<SqueezeBackward1>)

In [185]:
def toy_dl(mode=1, size=(64, 2, 2)):
    ope = {
        1 : torch.ones,
        0 : torch.zeros
    }
    dataset = ope[mode](*size)
    dataloader = torch_data.DataLoader(dataset, batch_size=4)
    
    return dataloader


class Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.layer = torch.nn.Conv2d(10, 10, 3)
    
    def training_step(self, batch, batch_idx):
        print(batch)

    def configure_optimizers(self):
        return torch.optim.Adam(self.layer.parameters())

In [186]:
dl1, dl0 = toy_dl(1), toy_dl(0)
model = Model()

In [187]:
trainer = pl.Trainer(max_epochs=1)
trainer.fit(model=model, train_dataloader=[dl1, dl0])


GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type   | Params
---------------------------------
0 | layer | Conv2d | 910   
---------------------------------
910       Trainable params
0         Non-trainable params
910       Total params
0.004     Total estimated model params size (MB)
Epoch 0:   0%|          | 0/16 [00:00<?, ?it/s] /usr/local/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
[tensor([[[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]]]), tensor([[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

   